In [69]:
embed_dim=100
G=4
#batch_size=80
number_of_negative_tokens = 5
n_epochs = 30
n_workers=64
queue_size = 64
padding_length=3
title_length_percentile=95
desc_length_percentile=90

import os, time, json,pandas as pd, numpy as np, re, string, random,sys
from collections import Counter,defaultdict
import Levenshtein


from keras.layers import Input,Lambda,Conv1D,GlobalMaxPool1D,concatenate,Dense,Dropout,PReLU,TimeDistributed,Embedding,Activation
from keras import Model
import keras.backend as K
import keras
from keras.utils import Sequence
import tensorflow as tf



df= pd.read_csv('allrecipes.csv')


# In[7]:


In [70]:
df.shape

(36429, 2)

In [71]:
df.WHO.value_counts()

1    12752
2    10950
3     6194
4     3017
0     1600
5     1412
6      444
7       60
Name: WHO, dtype: int64

In [3]:
#!pip3 install python-levenshtein
#!pip3 install setuptools

# strip punctuation and combine ingredients into one word separated by a spae

df['RECIPE'] = df['RECIPE'].str.replace('\d+', '')
df['RECIPE'] = df['RECIPE'].str.replace(' ', '')
df['RECIPE'] = df['RECIPE'].str.replace('#.,', ' ')
df['RECIPE'] = df['RECIPE'].str.replace('#.', '')

In [4]:
df['WHO'] = df['WHO'].replace({0:'zero', 1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 6: 'six', 7: 'seven'})

In [5]:
pd.options.display.max_colwidth= 500
df.head(4)

,RECIPE,WHO
0,mayonnaise parmigianoreggianocheese granulatedsugar romainelettuce baconbits,one
1,PhiladelphiaCreamCheese fineseasalt pecanhalves,one
2,hardboiledeggs wholemilk cakeflour fineseasalt vegetableoil,three
3,zucchini extra-virginoliveoil hotwater fineseasalt,three


In [6]:
mint=''.join(df['RECIPE'][0])

In [7]:
from sklearn.model_selection import train_test_split
#(X_train, X_test) = train_test_split(df2,test_size=0.20)


#with tf.device('/gpu:0'):
X_train, X_validation, X_test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

descTokenCounts = pd.Series(' '.join(df['RECIPE']).split())

descTokenCounts= descTokenCounts.value_counts().to_dict()

df= df.drop_duplicates()


df= df.dropna()


def remove_non_ascii_1(text):

    return ''.join([i if ord(i) < 128 else ' ' for i in text])

def remove_non_ascii_2(text):

    return re.sub(r'[^\x20-\x7E]+',' ', text)

vocabSize=100000
tokens = sorted(descTokenCounts.items(),key=lambda x:x[1],reverse=True)[:vocabSize]

tokens = [i[0] for i in tokens]
tokens.append('token_unknown')
token_to_index = {}
index_to_token = {}


df['WHO'] = ([remove_non_ascii_1(sentence) for sentence in df['WHO']])
df['WHO'] = ([remove_non_ascii_2(sentence) for sentence in df['WHO']])


df['RECIPE']= ([remove_non_ascii_1(sentence) for sentence in df['RECIPE']])


# In[13]:




df['RECIPE']= ([remove_non_ascii_2(sentence) for sentence in df['RECIPE']])



In [8]:
print(mint)

mayonnaise parmigianoreggianocheese granulatedsugar romainelettuce baconbits


In [9]:
model_name = 'allmy_jermaine_model1_jerm'


from sklearn.model_selection import train_test_split
#(X_train, X_test) = train_test_split(df2,test_size=0.20)


#with tf.device('/gpu:0'):
X_train, X_validation, X_test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])



with open('alltrain_data.json', 'wb') as f:
    f.write(X_train.to_json(orient='records', lines=True).encode('ascii', 'ignore'))

    
with open('alltest_data.json', 'wb') as g:
    g.write(X_test.to_json(orient='records', lines=True).encode('ascii', 'ignore'))


    
with open('allvalidation_data.json', 'wb') as t:
    t.write(X_validation.to_json(orient='records', lines=True).encode('ascii', 'ignore'))

def loadData(dataPath):
    data=[]
    start_time = time.time()
    with open(dataPath) as f:
        for line in f:
            data.append(line)
    loadTime = time.time() - start_time

    #print loadTime
    return data

training_data = loadData('alltrain_data.json')
validation_data = loadData('allvalidation_data.json') 
test_data = loadData('alltest_data.json') 


In [10]:
title_length= df['WHO'].map(len)
desc_length= df['RECIPE'].map(len)
titleCounts = df['WHO'].value_counts().to_dict()

charset=set(list(string.ascii_lowercase)+list(map(str,range(10)))+list("?$*!: @ ^ %- / <> ~ ` \\ [] {} \n = | \" _+#&-/\n %() ' '.;,"))
title_char_to_index_dict={c:i for i,c in enumerate(charset)}
title_index_to_char_dict={i:c for c,i in title_char_to_index_dict.items()}

In [11]:
print(title_char_to_index_dict)

{'.': 0, '\n': 1, '=': 2, ';': 3, 'p': 4, 'f': 5, '+': 6, ',': 7, 'j': 8, '0': 9, '2': 10, '?': 11, 's': 12, 'd': 13, '{': 14, '~': 15, 'b': 16, '*': 17, '%': 18, 'z': 19, 'w': 20, 'o': 21, 'y': 22, '8': 23, 'q': 24, '|': 25, '5': 26, '1': 27, 'a': 28, '!': 29, 'i': 30, '9': 31, '<': 32, 'n': 33, '6': 34, '$': 35, '`': 36, '}': 37, 'c': 38, '(': 39, '4': 40, ' ': 41, '_': 42, '-': 43, '^': 44, '@': 45, '7': 46, '\\': 47, '[': 48, '#': 49, 'e': 50, 'h': 51, 'g': 52, '>': 53, 'l': 54, "'": 55, 't': 56, 'r': 57, '/': 58, ')': 59, '"': 60, '&': 61, 'm': 62, ']': 63, 'x': 64, '3': 65, ':': 66, 'v': 67, 'u': 68, 'k': 69}


In [12]:



for i in range(len(tokens)):
    token_to_index[tokens[i]] = i+1
    index_to_token[i+1] = tokens[i] 


with open('allmodels_indices.json'.format(model_name),'w') as f:
    json.dump({'title_char_to_index_dict':title_char_to_index_dict,'title_index_to_char_dict':title_index_to_char_dict,              'token_to_index':token_to_index,'index_to_token':index_to_token},f)


    
title_max_len = np.ceil(np.percentile(title_length, title_length_percentile)).astype(int)

def title_char_tokenizer(entity,entity_max_len):
    preproced_entity=(" "*padding_length+entity)[:entity_max_len]
    padded_entity=preproced_entity+"".join([" "]*(entity_max_len-len(preproced_entity)))
    return [title_char_to_index_dict[i] for i in padded_entity]


## RECIPE tokenizer
## 0 padded in the end
desc_max_length = np.ceil(np.percentile(desc_length, desc_length_percentile)).astype(int)
def job_description_tokenizer(desc,entity_max_len):
    desc = re.sub("[,\/]",' ',desc).split()
    desc = [token_to_index[i] if i in token_to_index else token_to_index['token_unknown'] for i in desc]
    desc=(desc+[0]*entity_max_len)[:entity_max_len]
    return desc

In [13]:
with open ('allallmodels_model_params.json'.format(model_name),'w') as f:
    json.dump({'title_max_len':int(title_max_len),'desc_max_length':int(desc_max_length)},f)
    
## character embedding model for a list of titles
def char_embed_model_seq(input_layer,char_length,char_index_dict,entity_name,n_filters=512,embed_dim=embed_dim):
    '''
    Same as char_embed_model, but produces embeddings for a sequence of say, titles.
    Applies the same layers (as char_embed_model) to each member of the sequence
    '''    
    entity_onehot_layer=TimeDistributed(Lambda(lambda x: K.one_hot(x,len(char_index_dict)),output_shape=(char_length,len(char_index_dict))),name=entity_name+'_onehot_seq')(input_layer)
        
    conv7_layer=TimeDistributed(Conv1D(filters=n_filters,kernel_size=4,activation='relu'),name=entity_name+'_conv_7seq')(entity_onehot_layer)
    maxpool7_layer=TimeDistributed(GlobalMaxPool1D(),name=entity_name+'maxpool1seq')(conv7_layer)

    conv3_layer=TimeDistributed(Conv1D(filters=n_filters,kernel_size=2,activation='relu'),name=entity_name+'_conv_3seq')(entity_onehot_layer)
    maxpool3_layer=TimeDistributed(GlobalMaxPool1D(),name=entity_name+'maxpool2seq')(conv3_layer)

    pools_concat=keras.layers.concatenate([maxpool7_layer,maxpool3_layer],name=entity_name+'_maxpools_concatseq')
    #pools_dropped=(pools_concat)

    entity_dense_layer=TimeDistributed(Dense(n_filters*2,activation='relu'),name=entity_name+'_intermediate_denseseq')(pools_concat)
    
    dropout_layer=TimeDistributed(\Dropout(0.4),name=entity_name+"_droppedseq")(entity_dense_layer)
    
    entity_final_dense=TimeDistributed(Dense(embed_dim),name=entity_name+'_last_denseseq')(dropout_layer)
    entity_prelu_layer=TimeDistributed(PReLU(),name=entity_name+'_preluseq')(entity_final_dense)
    embed_norm_layer=TimeDistributed(Lambda(lambda x: x/(K.epsilon()+K.sqrt(K.batch_dot(x,x,axes=-1)))),name=entity_name+'_unit_norm_embedseq')(entity_prelu_layer)

    entity_embed_model=keras.models.Model(input_layer,embed_norm_layer,name=entity_name+"_embedding_model")
    return entity_embed_model




# In[25]:


## job describtion embedding - average word embedding (remove 0 padding)
def desc_embed_model(input_layer,vocab_size,entity_name,embed_dim=embed_dim):
    wordEmbed = Embedding(input_dim = vocab_size,output_dim = embed_dim, name='word_embedding')(input_layer) 
    
    ## get average embedding for desc (remove 0 padding)
    mask=Lambda(lambda x: K.cast(K.minimum(1,K.cast(x,'int32')),'float32'),name='mask')(input_layer)
    mask_summed=Lambda(lambda x: 1/(K.epsilon()+K.sum(x,axis=-1)),name='mask_summed')(mask)
    mask_normed=keras.layers.multiply([mask,mask_summed],name='mask_normed')
    desc_embedding_averaged=keras.layers.dot([wordEmbed,mask_normed],axes=1,name='desc_embedding_averaged')
    
    desc_embed_model=Model(input_layer,desc_embedding_averaged,name=entity_name+"_embedding_model")
    return desc_embed_model
# In[26]:


alpha=0.5
sampling_titles = titleCounts.keys()

sampling_titles= [k for k in titleCounts.keys()]


# In[27]:


sampling_titles_array = np.array(sampling_titles)


# In[28]:


sampling_prob_array = np.array(list(titleCounts.values()))
sampling_prob_array = np.power(sampling_prob_array,alpha)
sampling_prob_array = sampling_prob_array/sum(sampling_prob_array)


# In[29]:


def sampling_based_on_similarity(title,sub_sampling_titles,number_of_negative_tokens=5):
## Randomly sample sub_sampling_titles based on emperical distribution
## Randomly sample titles from the selected sub-titles based on Levenshtein similarity
    st = time.time()
    sampling_titles = np.array(sub_sampling_titles)

    sampling_prob = np.array([Levenshtein.ratio(title,i) for i in sub_sampling_titles])#Levenshtein.ratio(title,i) for i in sub_sampling_titles])
    sampling_prob = sampling_prob/sum(sampling_prob)
    
    samples=np.random.choice(sampling_titles,size=number_of_negative_tokens,p=sampling_prob)
  
    return samples.tolist()



In [17]:
class empirical_sampling_gen(Sequence):

    def __init__(self, data, batch_size,number_of_negative_tokens,sampling_titles,sampling_prob,sub_sampling_size):
        '''        
        batch_size: number of recipes to process per batch        
        The actual number of training samples varies as each resume is splitted into sub-sequences
        '''
        self.batch_size = batch_size
        self.input_data = data
        self.data_length = len(data)
        self.number_negative_tokens = number_of_negative_tokens
        self.sampling_titles = sampling_titles
        self.sampling_prob = sampling_prob
        self.sub_sampling_size = sub_sampling_size
    def __len__(self):
        return int(np.ceil(self.data_length / float(self.batch_size)))
    

    def __getitem__(self, idx):
        i = idx
        batch_size = self.batch_size 
        desc_input_batch=[]
        title_input_batch=[]
        y_pos_batch=[]
        y_neg_batch=[] 
        titles = []
        for k in range(i * batch_size, min((i + 1) * batch_size, self.data_length)):
            line = json.loads(self.input_data[k])
            desc = job_description_tokenizer(line['RECIPE'],desc_max_length)
            #desc = str(desc)
            titles.append(line['WHO'])
            title = title_char_tokenizer(line['WHO'],title_max_len)
            #title= str(title)
            desc_input_batch.append(desc)
            title_input_batch.append([title]*self.number_negative_tokens)
            y_pos_batch.append(1)
            y_neg_batch.append([0]*self.number_negative_tokens)  
            
        ## Sample negative titles for each batch all together              
        neg_titles = np.random.choice(self.sampling_titles, size=len(titles)*self.number_negative_tokens, p=self.sampling_prob)
        
        ## Resample for titles with length below median title length based on Levenshtein similarity
        sub_sampling_titles = np.random.choice(self.sampling_titles,p=self.sampling_prob,                                               size=self.sub_sampling_size*len(titles),replace=True)
        for i,t in enumerate(titles):
            if np.random.random()>0.5:
                sub_titles = sub_sampling_titles[i*self.sub_sampling_size:(i+1)*self.sub_sampling_size]
                samples = sampling_based_on_similarity(t,sub_titles)
                neg_titles[i*self.number_negative_tokens:(i+1)*self.number_negative_tokens]=samples
        
        neg_title_input_batch = [title_char_tokenizer(t,title_max_len) for t in neg_titles]
        neg_title_input_batch = [neg_title_input_batch[x:x+self.number_negative_tokens]                                  for x in range(0, len(neg_title_input_batch), self.number_negative_tokens)]
        
        
             
        ## Convert inputs and output to numpy arrays
        inputs = list(map(np.array,[desc_input_batch,title_input_batch,neg_title_input_batch]))
        outputs = list(map(np.array,[y_pos_batch,y_neg_batch]))     
        #return (inputs, outputs,titles,neg_titles) 
        #print(inputs)
        return (inputs, outputs) 
    

    def on_epoch_end(self):
        #'Shuffle data after each epoch'
        np.random.shuffle(self.input_data)



# In[ ]:


# In[31]:


batch_size=80


# In[32]:


trainGen = empirical_sampling_gen(training_data,batch_size,number_of_negative_tokens,                                  sampling_titles_array,sampling_prob_array,sub_sampling_size=2200)                                 



#len(title_char_to_index_dict)


# In[33]:



testGen = empirical_sampling_gen(test_data,batch_size,number_of_negative_tokens,                                    sampling_titles_array,sampling_prob_array,sub_sampling_size=1500)


# In[34]:


validateGen = empirical_sampling_gen(validation_data,batch_size,number_of_negative_tokens,                                    sampling_titles_array,sampling_prob_array,sub_sampling_size=2200)                                 


# In[35]:


target_title_input=Input(shape=(number_of_negative_tokens,title_max_len), dtype='int32', name='targe_title_input')
neg_title_input = Input(shape=(number_of_negative_tokens,title_max_len), dtype='int32', name='ne_title_input')
title_embedding_model = char_embed_model_seq(target_title_input,title_max_len,title_char_to_index_dict,"titlee")
target_title_embed = title_embedding_model(target_title_input)
target_title_embed = Lambda(lambda x:x[:,0,:],name='title_embedding_take_first_elemente')(target_title_embed)
neg_title_embed = title_embedding_model(neg_title_input)



# In[36]:


## Get resume job description embedding
desc_input = Input(shape=(desc_max_length,),name='des_input')
desc_embedding_model = desc_embed_model(desc_input,vocabSize+2,'des')
desc_embed = desc_embedding_model(desc_input)

## Calcuate dot product and loss
desc_target_title_dot = keras.layers.dot([target_title_embed,desc_embed],axes=-1,name='des_target_dot')
pos_title_logit = Activation('sigmoid',name='po_title_logit')(desc_target_title_dot)

desc_neg_title_dot = keras.layers.dot([neg_title_embed,desc_embed],axes=-1,name='des_neg_dot')
neg_title_logit = Activation('sigmoid',name='ne_title_logit')(desc_neg_title_dot)


In [19]:
embeddings = desc_embedding_model.get_weights()[0]
np.set_printoptions(suppress=True)
words_embeddings = {w:embeddings[idx].flatten() for w, idx in descTokenCounts.items()}

def recipe_emb(temp, words_embeddings):
    np.set_printoptions(suppress=True,
    formatter={'float_kind':'{:0.18f}'.format})
    count = 0
    v=1
    for w in temp.split():
        if w in words_embeddings:
            count +=1
            v+= ((words_embeddings[w]))
        #v= np.array(v)
    return v

In [35]:
words_embeddings['soup']

array([ 0.02448433,  0.03062416, -0.01681703,  0.02122458,  0.01187936,
       -0.02922964,  0.01593478, -0.03151363, -0.0379595 ,  0.02044037,
       -0.04961586,  0.02916038,  0.00845318,  0.00982299, -0.01208616,
       -0.02042514,  0.00065281,  0.03172705, -0.01481517, -0.00105858,
       -0.04450939, -0.02806205, -0.0399661 , -0.04933716,  0.02293475,
       -0.0012811 , -0.02535025, -0.02904861,  0.03837165,  0.00066642,
       -0.03472545, -0.00616946,  0.01452087,  0.01684133, -0.02522833,
        0.00843678,  0.0123084 , -0.0488718 ,  0.01559908, -0.00869668,
       -0.04525021, -0.01758415,  0.01584867, -0.01951263,  0.03611184,
       -0.03934405, -0.00283822,  0.02856929, -0.01540083,  0.02928745,
       -0.00649317, -0.01895791,  0.03905341, -0.01834018,  0.01103797,
       -0.0246984 , -0.04955833,  0.0464688 , -0.03899119, -0.03489087,
        0.00344292, -0.04812456,  0.00225431, -0.03226161, -0.04423666,
       -0.0343858 , -0.02604166,  0.01794333,  0.0194065 ,  0.01

In [41]:
def my_save_word2vec_format(fname, vocab, vectors, binary=True, total_vec=2):
    """Store the input-hidden weight matrix in the same format used by the original
    C word2vec-tool, for compatibility.

    Parameters
    ----------
    fname : str
    The file path used to save the vectors in.
    vocab : dict
    The vocabulary of words.
    vectors : numpy.array
    The vectors to be stored.
    binary : bool, optional
    If True, the data wil be saved in binary word2vec format, else it will be saved in plain text.
    total_vec : int, optional
    Explicitly specify total number of vectors
    (in case word vectors are appended with document vectors afterwards).
    
    """

    if not (vocab or vectors):
        raise RuntimeError("no input")
    if total_vec is None:
        total_vec = len(vocab)
    vector_size = vectors.shape[1]
    assert (len(vocab), vector_size) == vectors.shape
    with utils.smart_open(fname, 'wb') as fout:
        print(total_vec, vector_size)
        fout.write(utils.to_utf8("%s %s\n" % (total_vec, vector_size)))
    # store in sorted order: most frequent words at the top
        for word, row in vocab.items():
            if binary:
                row = row.astype(REAL)
                fout.write(utils.to_utf8(word) + b" " + row.tostring())
            else:
                fout.write(utils.to_utf8("%s %s\n" % (word, ' '.join(repr(val) for val in row))))

In [68]:
len(words_embeddings)

465

In [44]:
import gensim
from gensim.models import Word2Vec
from numpy import zeros, dtype, float32 as REAL, ascontiguousarray, fromstring
from gensim import utils

m = gensim.models.keyedvectors.Word2VecKeyedVectors(vector_size=100)
m.vocab = words_embeddings
m.vectors = np.array(list(words_embeddings.values()))
my_save_word2vec_format(binary=True, fname='train.bin', total_vec=len(words_embeddings), vocab=m.vocab, vectors=m.vectors)

465 100


In [45]:
m2 = gensim.models.keyedvectors.Word2VecKeyedVectors.load_word2vec_format('train.bin', binary=True)

In [65]:
#m2.most_similar(positive=['spaghetti', 'pasta'], negative=['ketchup'])
m2.similarity('ketchup', 'spaghetti')

/Users/jermainemarshall/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.046567295

In [519]:
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [520]:
df3= df[0:80000]
df3['embed_summed']=df3['RECIPE'].apply(lambda x: recipe_emb(x,words_embeddings))
df3.embed_summed= df3['embed_summed'].astype(str)
df3.embed_summed.str.replace('[','')
df3['embed_summed'] = df3['embed_summed'].str.replace('[', '')
df3['embed_summed'] = df3['embed_summed'].str.replace(']', '')
df3['embed_summed'] = df3['embed_summed'].str.replace('[\n\s]+', ',')
df3['embed_summed'] = df3['embed_summed'].str.replace(r'^,', '')
df4 = df3['embed_summed'].apply(lambda x: pd.Series(x.split(',')))
df3= df3[['RECIPE','WHO']].copy()
df3= df3.join(df4)


In [517]:
df3.to_csv('summed_allrecipe_average_embeddings.csv')

In [18]:
original_model=Model(inputs=[desc_input,target_title_input,neg_title_input],outputs=[pos_title_logit,neg_title_logit])


original_model.compile(optimizer='adam',metrics=['accuracy'],
                       loss=['binary_crossentropy','binary_crossentropy'],loss_weights=[1.,1.])                    

In [479]:
original_model.fit_generator(trainGen,verbose=1,validation_data=validateGen,epochs=5, steps_per_epoch= 30,
                             workers=n_workers,max_queue_size=queue_size,use_multiprocessing=False,shuffle=True)


Epoch 1/5
30/30 [==============================] - 362s 12s/step - loss: 1.3836 - po_title_logit_loss: 0.6895 - ne_title_logit_loss: 0.6941 - po_title_logit_acc: 0.6250 - ne_title_logit_acc: 0.4503 - val_loss: 1.3797 - val_po_title_logit_loss: 0.6844 - val_ne_title_logit_loss: 0.6952 - val_po_title_logit_acc: 0.6983 - val_ne_title_logit_acc: 0.4325
Epoch 2/5
15/30 [==============>...............] - ETA: 53s - loss: 1.3794 - po_title_logit_loss: 0.6817 - ne_title_logit_loss: 0.6978 - po_title_logit_acc: 0.7408 - ne_title_logit_acc: 0.3737

KeyboardInterrupt: 

In [16]:
open('all_recipe_model.json'.format(model_name), 'w').write(original_model.to_json())


# In[ ]:


original_model.save_weights('all_recipe_model.hdf5'.format(name=model_name))